In [2]:
from __future__ import print_function, division

# import os, sys
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir
from os.path import join
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils

In [3]:
path = r'D:\GitWork\dog_breed\data\processed'
print("Processed path: '{}'".format(ProcPath))

NameError: name 'ProcPath' is not defined

In [ ]:
csv_proc_breeds = 'breeds_processed.csv'

df_breeds = pd.read_csv(join(path, csv_proc_breeds))
print('\nBreeds info:'); print(df_breeds.info())
print('\nBreeds head:'); print(df_breeds.head())

# Get most popular breeds
NumClasses = 16

selected_breeds = list(df_breeds['breed'][:NumClasses] )
selected_bids = list(df_breeds['breed_id'][:NumClasses] )

print('\nSelected breeds: [\n  {}\n]'.format('\n  '.join(selected_breeds)))
print('\nSelected breed ids:', selected_bids)

df_breeds_selected = df_breeds[df_breeds['breed'].isin(selected_breeds)]
print(); print(df_breeds_selected)

In [12]:
fname = 'train_data.npz'

load_data = np.load(join(path, fname), allow_pickle=True)
df1 = pd.DataFrame.from_dict({item: load_data[item] for item in load_data.files})
print(df1.info())
print(df1.head())

col_names = df1.columns
print(); print(col_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8177 entries, 0 to 8176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   images  8177 non-null   object
 1   labels  8177 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 127.9+ KB
None
                                              images  labels
0  D:\GitWork\dog_breed\data\raw\train\000bec180e...      42
1  D:\GitWork\dog_breed\data\raw\train\001513dfcb...      72
2  D:\GitWork\dog_breed\data\raw\train\001cdf01b0...      94
3  D:\GitWork\dog_breed\data\raw\train\00214f311d...      50
4  D:\GitWork\dog_breed\data\raw\train\0021f9ceb3...     115

Index(['images', 'labels'], dtype='object')


In [19]:
fname = 'train_data.npz'

load_data = np.load(join(path, fname), allow_pickle=True)
column_names = load_data.files

df = pd.DataFrame(data=[load_data[x] for x in column_names]).T
df.columns = load_data.files

print(df.info())
print(); print(df.head)

df_delected = df[df['labels'].isin(selected_bids)]
print(); print(df_delected.info())
print(); print(df_delected.head)


8177
['D:\\GitWork\\dog_breed\\data\\raw\\train\\000bec180eb18c7604dcecc8fe0dba07.jpg'
 'D:\\GitWork\\dog_breed\\data\\raw\\train\\001513dfcb2ffafc82cccf4d8bbaba97.jpg'
 'D:\\GitWork\\dog_breed\\data\\raw\\train\\001cdf01b096e06d78e9e5112d419397.jpg'
 ...
 'D:\\GitWork\\dog_breed\\data\\raw\\train\\cc7ae3da3bebcc4acb10128078cdf29a.jpg'
 'D:\\GitWork\\dog_breed\\data\\raw\\train\\cc7af1c37c65fee4e036571c7f5b55f9.jpg'
 'D:\\GitWork\\dog_breed\\data\\raw\\train\\cc8ec46c8c1ff5337ac636a9c21abcc2.jpg']

 8177
[ 42  72  94 ... 109  64  22]

 1422
[8, 9, 12, 29, 48, 57, 70, 79, 80, 88]


In [25]:
fname = 'valid_data.npz'

load_data = np.load(join(path, fname), allow_pickle=True)
column_names = load_data.files

df = pd.DataFrame(data=[load_data[x] for x in column_names]).T
df.columns = load_data.files

print(df.info())
print(); print(df.head)

df_delected = df[df['labels'].isin(selected_bids)]
print(); print(df_delected.info())
print(); print(df_delected.head)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   images  2045 non-null   object
 1   labels  2045 non-null   object
dtypes: object(2)
memory usage: 32.1+ KB
None

<bound method NDFrame.head of                                                  images labels
0     D:\GitWork\dog_breed\data\raw\train\cc93915e06...     37
1     D:\GitWork\dog_breed\data\raw\train\cc964d3bf1...     60
2     D:\GitWork\dog_breed\data\raw\train\cc97041986...     83
3     D:\GitWork\dog_breed\data\raw\train\cc99de39a1...     22
4     D:\GitWork\dog_breed\data\raw\train\cc9b4190a7...     33
...                                                 ...    ...
2040  D:\GitWork\dog_breed\data\raw\train\ffd25009d6...     93
2041  D:\GitWork\dog_breed\data\raw\train\ffd3f636f7...     36
2042  D:\GitWork\dog_breed\data\raw\train\ffe2ca6c94...     10
2043  D:\GitWork\dog_breed\data\raw\train\f